In [ ]:
"""
This is an example of RAG engine using retrivial to scan through a company's past 12 months earning transcript. 
Then user can ask relevant questions.
It uses google/flan-t5-base or google/flan-t5-large as base model and use earning trasnript PDF as context for retrivial.
"""

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

/Users/zhouyu/Documents/notebook/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/zhouyu/Documents/notebook/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-17 10:09:27.229615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None

Settings.chunk_size = 256
Settings.chunk_overlap = 25
documents = SimpleDirectoryReader("stock-articles").load_data()

LLM is explicitly disabled. Using MockLLM.


In [3]:
print(len(documents))
for doc in documents:
    # reduce document size, remove operator related comments     
    if "Operator" in doc.text:
        documents.remove(doc)
        continue


print(len(documents))


126
89


In [19]:

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

# set number of docs to retreive
top_k = 5

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)


In [26]:
# query documents
comment = "What is going on with HeyDude's margin?"
response = query_engine.query(comment)

# 
context = "Context:\n"
for i in range(top_k):
    # This step enrich the context by retriving information from the articles
    context = context + response.source_nodes[i].text + "\n\n"
print(context)


Context:
Can you talk about that effort, the timing, the rationale and so forth?
Anne Mehlman -- Executive Vice President, Chief Financial O cer
Sure.
Andrew Rees -- Director and Chief Executive O cer
Yeah. So from a margin -- we're not anticipating incremental margin pressure from the
lower revenue, some of the revenue reduction for HEYDUDE. I think we -- the margin is
sort of playing out our pricing, promotional strategy, our channel mix is playing out
pretty much where we thought it would be. And obviously, the reduction is really
coming from wholesale revenues and so as our DTC revenues are obviously higher
margin than wholesale revenues, we don't see a margin reduction.
We think our margin guide is good for HEYDUDE, and I'll let Anne talk about the ERP.

which supports that overall gross margin. We also have seen a little bit of input cost
pressure relief on the Crocs side.
And so we've seen our costs as we negotiate those kind of in the  rst quarter come
down. And so we've taken 

In [6]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-base"
# model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [27]:
instructions_string = """
You are a very smart financial analyst. We are trying to extract relevant information about a publicly traded stock in the US. 
Please respond to the following comment."""

prompt_template = lambda comment: f"{instructions_string}\n{comment}"



prompt = prompt_template(comment)
print(prompt)

model.eval()

# Tokenize prompts, return in PyTorch format
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=100)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


You are a very smart financial analyst. We are trying to extract relevant information about a publicly traded stock in the US. 
Please respond to the following comment.
What is going on with HeyDude's margin?
It is a stock that has a high price.


In [28]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""
You are a very smart financial analyst. We are trying to extract relevant information about a publicly traded stock in the US. 

{context}
Based on the context provided, please respond to the following comment. Make sure to use the context above. 

{comment}

"""

prompt = prompt_template_w_context(context, comment)
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=150)




You are a very smart financial analyst. We are trying to extract relevant information about a publicly traded stock in the US. 

Context:
Can you talk about that effort, the timing, the rationale and so forth?
Anne Mehlman -- Executive Vice President, Chief Financial O cer
Sure.
Andrew Rees -- Director and Chief Executive O cer
Yeah. So from a margin -- we're not anticipating incremental margin pressure from the
lower revenue, some of the revenue reduction for HEYDUDE. I think we -- the margin is
sort of playing out our pricing, promotional strategy, our channel mix is playing out
pretty much where we thought it would be. And obviously, the reduction is really
coming from wholesale revenues and so as our DTC revenues are obviously higher
margin than wholesale revenues, we don't see a margin reduction.
We think our margin guide is good for HEYDUDE, and I'll let Anne talk about the ERP.

which supports that overall gross margin. We also have seen a little bit of input cost
pressure reli

In [29]:
## significantly better improvements. 
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

HEYDUDE brand adjusted gross margin was 47.9% or 510 basis points higher than prior year, driven primarily by freight, favorable channel mix, and pricing.
